# BERT - Out of the Box

In this notebook, we will test the performance of an out-of-the-box BERT model on CommonsenseQA. I follow the tutorial here: https://github.com/jalammar/jalammar.github.io/blob/master/notebooks/bert/A_Visual_Notebook_to_Using_BERT_for_the_First_Time.ipynb

I've implemented the Hugginface transformers library. 

I referred to the Commonsense QA repo and code to understand how the authors of this work establiahsed their baseline using BERT. This is the link to their code: https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

From this repo (README): https://github.com/jonathanherzig/commonsenseqa

Their work is far more advanced and complicated than maybe what I want to do at this time. But I refer to their work to understand the set up.

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score

import warnings

import json
from pandas.io.json import json_normalize
warnings.filterwarnings('ignore')


from transformers import BertTokenizer, TFBertModel, BertConfig

import tensorflow as tf
from tensorflow import keras

from datetime import datetime

import time 
configuration = BertConfig() 
from IPython.display import Image 

In [ ]:
print(tf.__version__) 

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
print("Num CPUs Available: ",
      len(tf.config.experimental.list_physical_devices('CPU')))


## Import dataset

It's in the dataset folder.

In [ ]:
def load_data(file):
    lines = []
    with open(file, 'rb') as json_file:
        for json_line in json_file:
            lines.append(json.loads(json_line))
        data = json_normalize(lines)
        data.columns = data.columns.map(lambda x: x.split(".")[-1])
    return data
# os.chdir('w266-commonsenseqa/BERT_oob)
train = load_data('../dataset/train_rand_split.jsonl')
dev = load_data('../dataset/dev_rand_split.jsonl')
train.head()

In [ ]:
# Use DistilBert because kernel keeps crashing 

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
lab_order = {"A": 0, "B":1, "C":2, "D":3, "E":4}

class InputExample(object):
    """A single multiple choice question."""
    # This class is adapted from https://github.com/jonathanherzig/commonsenseqa/blob/master/bert/run_commonsense_qa.py

    def __init__(
            self,
            qid,
            question,
            answer,
            label):
        """Construct an instance."""
        self.qid = qid
        self.question = question  # e.g., 'The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?'
        self.answer = answer      # e.g., "ignore" if choice label is A 
        self.label = label        # e.g., If correct answer, 1. Otherwise 0. 
        
    def __str__(self):
        return "QUESTION: {}\nANSWER  : {}\nLABEL   : {}".format(self.question, self.answer, self.label)
    
    
def create_example(row, choice_num):
    qid = row.id
    
    # Question: Just take it from stem 
    question = row.stem
    
    # Answer choice 
    label = int(row["answerKey"] == choice_num)  # If the answer key is equal to the answer choice number, mark 1 
    answer = row["choices"][lab_order[choice_num]]["text"]         # actual ans text 
    
    return InputExample(qid, question, answer, label) 
    
    
def process_examples(data):
    examples = []
    labels = []
    questions = []
    anscands = []
    
    
    for index, row in data.iterrows(): 
        for letter in lab_order.keys():
            example = create_example(row, letter)
            examples.append(example)
            
            questions.append(example.question)
            anscands.append(example.answer)
            labels.append(example.label)
    labels = np.array(labels)
        
    encoded_example = tokenizer(questions, anscands, padding=True, truncation=True, return_tensors='tf')
    
    # Make the encoded example a dictionary of np arrays
    for key in encoded_example:
        encoded_example[key] = np.array(encoded_example[key])
        
    clean_encoded_example = [
        encoded_example["input_ids"],
        encoded_example["attention_mask"],
        encoded_example["token_type_ids"]
    ]
            
    return examples, clean_encoded_example, labels


In [ ]:
tiny_train_eg, bert_inputs_train_tiny, bert_labs_train_tiny = \
process_examples(train.iloc[0:40])

Process dev and training data

In [ ]:
dev_eg, dev_encoded_eg, dev_labs = process_examples(dev)

In [ ]:
train_eg, train_encoded_eg, train_labs = process_examples(train)

In [11]:
print(train_eg[0])

QUESTION: The sanctions against the school were a punishing blow, and they seemed to what the efforts the school had made to change?
ANSWER  : ignore
LABEL   : 1


### Load Model

The models were trained and saved as file. 

In [20]:
BertClassifierModel = tf.keras.models.load_model('models/bert_classification_1018')

TypeError: The two structures don't have the same nested structure.

First structure: type=list str=[TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/2')]

Second structure: type=dict str={'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='inputs/input_ids')}

More specifically: The two namedtuples don't have the same sequence type. First structure type=list str=[TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/0'), TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/1'), TensorSpec(shape=(None, 48705), dtype=tf.int32, name='inputs/2')] has type list, while second structure type=dict str={'input_ids': TensorSpec(shape=(None, 5), dtype=tf.int32, name='inputs/input_ids')} has type dict
Entire first structure:
[., ., .]
Entire second structure:
{'input_ids': .}

In [54]:
('models/bert_classification_1018_3epochs', save_format='tf')


Epoch 1/2
1523/1523 [==============================] - 1414s 929ms/step - loss: 0.5026 - accuracy: 0.8000
Epoch 2/2
1523/1523 [==============================] - 1412s 927ms/step - loss: 0.5023 - accuracy: 0.8000
Execution duration in minutes: 47.13618437846502
INFO:tensorflow:Assets written to: models/bert_classification_1018_3epochs/assets
